In [77]:
import numpy as np
import rebound
import pandas as pd
import assist
import astroquery
from astroquery.jplhorizons import Horizons
from astropy.time import Time
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import math
from astropy.io import fits
from astropy.wcs import WCS
%matplotlib inline

import os

In [78]:
def astroquery_lookup(name,epoch,location='@0',ref='earth'):
    spkid = str(2000000+int(name)) #go from mpc elements to an SPK ID
    obj = Horizons(id=spkid, id_type='designation', location=location,epochs=epoch) #default location
    state_vec = np.array(list(np.array(obj.vectors(refplane=ref,cache=False)['x','y','z','vx','vy','vz'])[0]))
    return state_vec

def astroquery_ephem(id,epoch,id_type=None,location='399'):
    #default location is gaia
    #units are RA,DEC and AU
    #epoch is UTC for ephmereis queries TBD for element or vector
    epochutc = Time(epoch,format='jd',scale='tdb').utc.value
    objutc = Horizons(id=id,epochs=epochutc,id_type=id_type,location=location) #default location, must give UTC for ephemeris
    ephem = objutc.ephemerides(extra_precision=True,cache=False)[['RA','DEC','delta']].as_array()[0]
    return np.array(list(ephem))

def cart_to_spherical(X,Y,Z):
	#default location is solar system barycenterrad2deg = 180/np.pi
	rad2deg = 180/np.pi
	a = np.arctan2(Y,X)*rad2deg #this can be negative
	r = np.sqrt(X**2 + Y**2 + Z**2)
	d = np.arcsin(Z/r)*rad2deg
	return np.array([np.mod(a,360), d, r]).T

In [79]:
#Zach psuedocode to do integration funciton. Check for bugs! 
def integration_function(ephem,asteroid_initial_state,jd_finals):
    #jd_final = 2414819.68100 #2414819.68659
    t_finals = jd_finals - ephem.jd_ref
    t_initial = jd_initial - ephem.jd_ref

    state = asteroid_initial_state   
    target = rebound.Particle(x=state[0],y=state[1],z=state[2],vx=state[3],vy=state[4],vz=state[5])

    sim = rebound.Simulation()
    sim.add(target) 
    sim.t = t_initial 
    sim.ri_ias15.min_dt = 0.001
    extras = assist.Extras(sim, ephem)
    extras.gr_eih_sources = 11 # Turn on GR for star and all planets

    N_samples = len(t_finals)
    pos_target = np.zeros((N_samples, 3))
    earth_pos = np.zeros((N_samples, 3))
    for i, t in enumerate(t_finals):
        extras.integrate_or_interpolate(t)
        pos_target[i] = sim.particles[0].xyz
        earth_pos[i] = ephem.get_particle("earth", t).xyz
        
    return pos_target, earth_pos


In [80]:
def get_coords_from_file(target_name, filename="initial_states.csv"):
    f = open(filename, newline="", encoding="utf-8")
    for line in f:
        line = line.split(",")
        if int(line[0]) == target_name:
            return [line[1], line[2], line[3], line[4], line[5], line[6]]

In [81]:
JD_file = "sorted_jds.csv"
julian_dates = np.loadtxt(JD_file, delimiter=',', dtype=float)
ordering = julian_dates[:,0]
julian_dates = julian_dates[:,1]
print(f"Data type: {type(julian_dates)}, Shape: {julian_dates.shape}")  #this is good practice
print(f"First 5 JDs: {julian_dates[:5]}")

Data type: <class 'numpy.ndarray'>, Shape: (423697,)
First 5 JDs: [2449751.49040509 2448847.86493056 2448847.84479167 2448847.83055556
 2448847.76006944]


In [82]:
def get_coords_from_file(target_name, filename="initial_states.csv"):
    f = open(filename, newline="", encoding="utf-8")
    for line in f:
        line = line.split(",")
        if int(line[0]) == target_name:
            return [float(line[1]), float(line[2]), float(line[3]), float(line[4]), float(line[5]), float(line[6])]

In [83]:
c=173.14463271362928
jd_initial = 2451545.0

def get_coords(target_name,initial_jd_finals1):
    asteroid_initial_state=get_coords_from_file(target_name)   #rather than use the states I'll just record them
    ephem = assist.Ephem("data/linux_p1550p2650.440", "data/sb441-n16.bsp")

    coords_asteroid1, coords_earth1 = integration_function(ephem,asteroid_initial_state,initial_jd_finals1) #coordinates at jd final
    dt1 = np.linalg.norm(coords_asteroid1 - coords_earth1,axis=1)/c

    initial_jd_finals2 = initial_jd_finals1 - dt1
    coords_asteroid2, coords_earth2 = integration_function(ephem,asteroid_initial_state,initial_jd_finals2) #coordinates at jd final
    dt2 = np.linalg.norm(coords_asteroid2 - coords_earth1,axis=1)/c

    initial_jd_finals3 = initial_jd_finals1 - dt2
    coords_asteroid3, coords_earth3 = integration_function(ephem,asteroid_initial_state,initial_jd_finals3) #coordinates at jd final
    dt3 = np.linalg.norm(coords_asteroid3 - coords_earth1,axis=1)/c

    initial_jd_finals4 = initial_jd_finals1 - dt3
    coords_asteroid4, coords_earth4 = integration_function(ephem,asteroid_initial_state,initial_jd_finals4) #coordinates at jd final
    dt4 = np.linalg.norm(coords_asteroid4 - coords_earth1,axis=1)/c

    calculated_coords = cart_to_spherical(*(coords_asteroid4 - coords_earth1).T)
    return calculated_coords

In [84]:
alltargets = pd.read_csv('targets.csv')['desgs'].values

In [85]:
#make directory for the files:
import os
path = './results/'
os.makedirs(path, exist_ok=True)


In [86]:
import time
for mpcnum in alltargets: #loop over files
    print("Working on",mpcnum)
    start =time.time()
    coords  = get_coords(mpcnum,julian_dates)
    #save coordinates to file
    np.savez_compressed(path+str(mpcnum)+'.npz',coords)
    print("elasped=", time.time()- start, " seconds")


Working on 1
elasped= 8.231153726577759  seconds
Working on 2
elasped= 9.126529932022095  seconds
Working on 3
elasped= 9.245004177093506  seconds
Working on 4
elasped= 8.003804206848145  seconds
Working on 5
elasped= 9.00631594657898  seconds
Working on 6
elasped= 9.451272010803223  seconds
Working on 7
elasped= 9.048254013061523  seconds
Working on 8
elasped= 9.079535961151123  seconds
Working on 9
elasped= 8.91602611541748  seconds
Working on 10
elasped= 8.293902158737183  seconds
Working on 11
elasped= 8.81493330001831  seconds
Working on 12
elasped= 9.044578075408936  seconds
Working on 13
elasped= 8.872231006622314  seconds
Working on 14
elasped= 8.730147123336792  seconds
Working on 15
elasped= 8.349678993225098  seconds
Working on 16
elasped= 8.936702966690063  seconds
Working on 17
elasped= 8.843459129333496  seconds
Working on 18
elasped= 8.886151790618896  seconds
Working on 19
elasped= 8.89671516418457  seconds
Working on 20
elasped= 8.841495990753174  seconds
Working on 21